In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [57]:
E_B = 100
P_Battery_Max = 50
init_SOC=0.5
SOC_UP =  soc_upper_bound = 0.9
SOC_FLOOR = soc_floor_bound = 0.1
eta_Bc=eta_Bd=0.95
Cost_Battery = P_Battery_Max*800+E_B*1800

Battery_lifetime = 10*365 #忽略闰年
Cost_Battery_daily = Cost_Battery/(Battery_lifetime)

# 读取数据
data_A = pd.read_csv("data/园区A.csv")
data_B = pd.read_csv("data/园区B.csv")
data_C = pd.read_csv("data/园区C.csv")

P_user_A = data_A["用电量"]
P_user_B = data_B["用电量"]
P_user_C = data_C["用电量"]
P_PV_A = data_A["光伏供电量"]
P_WG_B = data_B["风电供电量"]
P_PV_C = data_C["光伏供电量"]
P_WG_C = data_C["风电供电量"]


In [58]:
# C
def setSOC(SOC,t,value):
    SOC[t]=value
def getSOC(SOC,t):
    if t==-1:
        return SOC[-1]
    return SOC[t]
def gamma_B(P_B):
    if P_B<0:
        return eta_Bc
    else:
        return 1/eta_Bd
def get_P_B(Delta_P):#限制充放电功率
    P_Bdmax = P_Bcmax = P_Battery_Max
    if Delta_P>-P_Bdmax and Delta_P<=P_Bcmax:
        return Delta_P
    elif Delta_P>P_Bcmax:
        return P_Bcmax
    else:
        return -P_Bdmax

# 放电时为正，充电时为负
def cal_P_B(SOC,P_Battery_Max,Delta_P,t):
    P_B=get_P_B(Delta_P) 
    print(SOC)
    SOC_last = getSOC(SOC,t) 
    gamma_B_value = gamma_B(P_B)
    SOC_new = SOC_last-gamma_B_value*P_B*1/E_B
    
    # 限制充放电容量
    if SOC_new > SOC_UP:
        SOC_new = SOC_UP
    elif SOC_new < SOC_FLOOR:
        SOC_new = SOC_FLOOR
    P_B_Battery = (SOC_last-SOC_new)*E_B
    P_B_use =(SOC_last-SOC_new)*E_B/gamma_B(P_B)/1

    if(P_B_use>0):
        print(f"外部放电{P_B_use}kW,消耗电池{P_B_Battery}")
    else:
        print(f"外部充电{P_B_use}kW,补充电池{P_B_Battery}")
    return P_B_use,P_B_Battery
    
def find_lack(t):
    data = P_PV_C+P_WG_C-P_user_C
    for i in range(t+1,len(data)):
        if data[i] < 0:
            return i
    return -1

def wait_sun(P_wind,Battery,SOC,t):
    j = t
    n = find_lack(t)
    lack = P_user_C[n]-P_PV_C[n]-P_WG_C[n]
    extra=0
    for k in range(j+1,n):
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,(P_PV_C[k])+P_WG_C[k]-P_user_C[k],t)
        extra+=abs(P_B_use)
    B_empty = (SOC_UP-getSOC(SOC,t))*E_B
    if(lack-extra>B_empty):
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t)
        P_wind+=P_B_use
        Battery-=P_B_Battery 
        C_wind=-P_B_use 
        return P_wind,C_wind,Battery 
    while(n<=365*24):
        j = n+1
        n = find_lack(n)
        lack = P_user_C[n]-P_PV_C[n]-P_WG_C[n]
        extra=0
        for k in range(j+1,n):
            P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,(P_PV_C[k])+P_WG_C[k]-P_user_C[k],t)
            extra+=abs(P_B_use)
        B_empty = (SOC_UP-getSOC(SOC,t))*E_B
        if(lack-extra>B_empty):
            P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t) 
            P_wind+=P_B_use
            Battery-=P_B_Battery 
            C_wind=-P_B_use 
    return P_wind,C_wind,Battery 


def debug_Battery(Battery,SOC,t,debug):
    if not debug :
        return 
    print(f"电池电量：{Battery},{SOC[t]}")


# Battery = init_SOC*E_B or 0
def Process(SOC,Battery,P_user, P_light, P_wind,label,t,debug=False):
    
    print(f"现在是{t}:00,电池电量：{Battery}")
    
    # 光
    C_light=0
    C_wind=0 
    C_net=0
    if P_light > P_user:
        C_light = P_user
        P_user = 0
        P_light -= C_light
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_light,t)
        # charge
        P_light-=P_B_use 
        Battery+=P_B_Battery
        debug_Battery(Battery,SOC,t,debug)
    else:
        C_light = P_light
        P_user -= P_light
    
    if P_user == 0:
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net,Battery
    
    # 电池
    # discharge
    if(debug):
        print("自发电不足，动用电池放电")
    P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,P_user,t) #放电时为正，充电时为负
    Battery -= P_B_Battery 
    debug_Battery(Battery,SOC,t,debug)

    P_user -=P_B_use 
    if P_user==0:
        P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t) 
        P_wind+=P_B_use
        Battery-=P_B_Battery 
        debug_Battery(Battery,SOC,t,debug)
        C_wind=-P_B_use 
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net,Battery
    
    # 风
    if P_wind > P_user:
        C_wind = P_user
        P_user = 0
        P_wind-=C_wind

        if label == 'B':
            P_B_use,P_B_Battery = cal_P_B(SOC,P_Battery_Max,-P_wind,t)
            C_wind -= P_B_use
            P_wind +=P_B_use 
            Battery-=P_B_Battery  
            debug_Battery(Battery,SOC,t,debug)
            
        if label == 'C':
            # 额外考虑
            P_wind,C_wind,Battery = wait_sun(P_wind,Battery,SOC,t)#################
            debug_Battery(Battery,SOC,t,debug)
            
        SOC.append(Battery/E_B)
        return C_light, C_wind, C_net,Battery

        
    else:
        C_wind = P_wind
        P_user -= P_wind
    
    if P_user > 0:
        C_net = P_user
        P_user = 0
    # SOC[t]=Battery/E_B
    SOC.append(Battery/E_B)
    
    return C_light, C_wind, C_net,Battery

In [64]:
# C
Battery = init_SOC*E_B
cost_C = []
sum_C = Cost_Battery_daily
SOC=[init_SOC]
for t in range(0,24):
    C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_C[t], 0,P_WG_C[t],"C",t,debug=True) 
    # print(C_light, C_wind, C_net)
    cost_C.append(C_light*0.4+C_wind*0.5+C_net*1) 
    print(cost_C[t])
    sum_C+=cost_C[t]
print(cost_C)
# print(sum(cost_C))
print(sum_C)

# bug:
# 电池只有0.1时还在放电，应该终止

现在是0:00,电池电量：50.0
自发电不足，动用电池放电
[0.5]
外部放电38.0kW,消耗电池40.0
电池电量：10.0,0.5
227.4
现在是1:00,电池电量：10.0
自发电不足，动用电池放电
[0.5, 0.1]
外部充电0.0kW,补充电池0.0
电池电量：10.0,0.1
237.625
现在是2:00,电池电量：10.0
自发电不足，动用电池放电
[0.5, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
电池电量：10.0,0.1
208.025
现在是3:00,电池电量：10.0
自发电不足，动用电池放电
[0.5, 0.1, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
电池电量：10.0,0.1
247.225
现在是4:00,电池电量：10.0
自发电不足，动用电池放电
[0.5, 0.1, 0.1, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
电池电量：10.0,0.1
153.1
现在是5:00,电池电量：10.0
自发电不足，动用电池放电
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
电池电量：10.0,0.1
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1]
外部充电-50.0kW,补充电池-47.5
电池电量：57.5,0.1
25.0
现在是6:00,电池电量：57.5
自发电不足，动用电池放电
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.575]
外部放电45.125kW,消耗电池47.5
电池电量：10.0,0.575
164.22500000000002
现在是7:00,电池电量：10.0
自发电不足，动用电池放电
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.575, 0.1]
外部充电0.0kW,补充电池0.0
电池电量：10.0,0.1
192.65
现在是8:00,电池电量：10.0
自发电不足，动用电池放电
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.575, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
电池电量：10.0,0.1
285.75
现在是9:00,电池电量：10.0
自发电不足，动用电池放电
[0.5, 0.1, 0.

In [62]:
# A
Battery = init_SOC*E_B
SOC=[init_SOC]
cost_A = []
sum_A = Cost_Battery_daily
for t in range(0,24):
    C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_A[t], P_PV_A[t], 0,"A",t)
    print(C_light, C_wind, C_net)
    cost_A.append(C_light*0.4+C_wind*0.5+C_net*1) 
    sum_A+=cost_A[t] 
    print(cost_A[t])


# sum_A = sum(cost_A)
print(cost_A)
print(sum_A)

现在是0:00,电池电量：50.0
[0.5]
外部放电38.0kW,消耗电池40.0
0.0 0 237.0
237.0
现在是1:00,电池电量：10.0
[0.5, 0.1]
外部充电0.0kW,补充电池0.0
0.0 0 275.0
275.0
现在是2:00,电池电量：10.0
[0.5, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
0.0 0 277.0
277.0
现在是3:00,电池电量：10.0
[0.5, 0.1, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
0.0 0 310.0
310.0
现在是4:00,电池电量：10.0
[0.5, 0.1, 0.1, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
0.0 0 310.0
310.0
现在是5:00,电池电量：10.0
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
0.0 0 293.0
293.0
现在是6:00,电池电量：10.0
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
0.0 0 293.0
293.0
现在是7:00,电池电量：10.0
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
4.35 0 375.65
377.39
现在是8:00,电池电量：10.0
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
外部充电0.0kW,补充电池0.0
226.95 0 148.05
238.83
现在是9:00,电池电量：10.0
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
外部充电-50.0kW,补充电池-47.5
281 0 0
112.4
现在是10:00,电池电量：-37.5
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, -0.375]
外部充电-50.0kW,补充电池-47.5
447 0 0
178.8
现在是11:00,电池电量：-85.0
[0.5, 0.1, 0.1, 0.1, 0.1, 0.1,

In [63]:
# B
Battery = init_SOC*E_B
SOC=[init_SOC]
cost_B = []
sum_B = Cost_Battery_daily
for t in range(0,24):
    C_light, C_wind, C_net,Battery = Process(SOC,Battery,P_user_B[t], 0,P_WG_B[t],"B",t)
    print(C_light, C_wind, C_net)
    cost_B.append(C_light*0.4+C_wind*0.5+C_net*1) 
    print(cost_B[t])
    sum_B+=cost_B[t]

# sum_B = sum(cost_B)
print(cost_B)
print(sum_B)

现在是0:00,电池电量：50.0
[0.5]
外部放电38.0kW,消耗电池40.0
[0.5]
外部充电-27.099999999999998kW,补充电池-25.744999999999997
0 230.1 0
115.05
现在是1:00,电池电量：35.745
[0.5, 0.35745]
外部放电24.457749999999997kW,消耗电池25.744999999999997
[0.5, 0.35745]
外部充电-49.99999999999999kW,补充电池-47.49999999999999
0 278.54224999999997 0
139.27112499999998
现在是2:00,电池电量：57.49999999999999
[0.5, 0.35745, 0.575]
外部放电45.125kW,消耗电池47.5
[0.5, 0.35745, 0.575]
外部充电-12.925000000000006kW,补充电池-12.278750000000006
0 296.8 0
148.4
现在是3:00,电池电量：22.27875
[0.5, 0.35745, 0.575, 0.2227875]
外部放电11.6648125kW,消耗电池12.278749999999999
[0.5, 0.35745, 0.575, 0.2227875]
外部充电-50.0kW,补充电池-47.5
0 353.3351875 0
176.66759375
现在是4:00,电池电量：57.5
[0.5, 0.35745, 0.575, 0.2227875, 0.575]
外部放电45.125kW,消耗电池47.5
[0.5, 0.35745, 0.575, 0.2227875, 0.575]
外部充电-34.21052631578948kW,补充电池-32.50000000000001
0 279.0855263157895 0
139.54276315789474
现在是5:00,电池电量：42.50000000000001
[0.5, 0.35745, 0.575, 0.2227875, 0.575, 0.42500000000000004]
外部放电30.875000000000007kW,消耗电池32.50000000000001
[0.5,

In [52]:
E_B = 100
P_Battery_Max = 50


# 读取数据
data_A = pd.read_csv("data/园区A.csv")
data_B = pd.read_csv("data/园区B.csv")
data_C = pd.read_csv("data/园区C.csv")

df_A = pd.DataFrame(columns=['E_B', 'P_Battery_Max', 'cost'])
df_B = pd.DataFrame(columns=['E_B', 'P_Battery_Max', 'cost'])
df_C = pd.DataFrame(columns=['E_B', 'P_Battery_Max', 'cost'])


while(E_B<=1000):
    P_Battery_Max = 50
    while(P_Battery_Max<=1000):
        # 初始化变量
        Battery = 0
        init_Battery_capacity_floor_bound = E_B * soc_floor_bound
        init_Battery_capacity_upper_bound = E_B * soc_upper_bound

        cost_of_Battery=(800*P_Battery_Max+1800*E_B)/3650

        # 处理数据
        #print('E_B:',E_B,'P_Battery_Max:',P_Battery_Max)
        # A处理
        n_rows = data_A.shape[0]

        # 创建一个长度与data_A行数相同的空列向量
        A_light = np.zeros((n_rows,1))
        A_wind = np.zeros((n_rows,1))
        A_net = np.zeros((n_rows,1))

        # 循环遍历data_A的每一行
        for index, row in data_A.iterrows():
            P_user = row['用电量']
            P_light = row['光伏供电量']
            P_wind = 0
            A_light[index],A_wind[index],A_net[index]=Process(P_user, P_light, P_wind,data_A,'A',index)

        # print(A_light)
        # print(A_wind)
        # print(A_net)
        cost_A=1*A_net+0.4*A_light
        #print('A成本:',sum(cost_A)+cost_of_Battery)


        #B处理
        n_rows = data_B.shape[0]

        # 创建一个长度与data_B行数相同的空列向量
        B_light = np.zeros((n_rows,1))
        B_wind = np.zeros((n_rows,1))
        B_net = np.zeros((n_rows,1))

        # 循环遍历data_B的每一行
        for index, row in data_B.iterrows():
            P_user = row['用电量']
            P_wind = row['风电供电量']
            P_light = 0
            B_light[index],B_wind[index],B_net[index]=Process(P_user, P_light, P_wind,data_B,'B',index)

        #print(B_light)
        #print(B_wind)
        #print(B_net)
        cost_B=1*B_net+0.5*B_wind
        #print('B风电购入',sum(B_wind))
        #print('B电网购入',sum(B_net))
        #print('B成本:',sum(cost_B)+cost_of_Battery)

        # C处理
        n_rows = data_C.shape[0]

        # 创建一个长度与data_C行数相同的空列向量
        C_light = np.zeros((n_rows,1))
        C_wind = np.zeros((n_rows,1))
        C_net = np.zeros((n_rows,1))

        data_C['Curt_C'] = data_C.iloc[:, 1] + data_C.iloc[:, 2] - data_C.iloc[:, 0]
        #print(data_C['Curt_C'])
        # 循环遍历data_C的每一行
        for index, row in data_C.iterrows():
            P_user = row['用电量']
            P_light = row['光伏供电量']
            P_wind = row['风电供电量']
            C_light[index],C_wind[index],C_net[index]=Process(P_user, P_light, P_wind,data_C,'C',index)

        #print(C_light)
        #print(C_wind)
        #print(C_net)
        cost_C=1*C_net+0.4*C_light+0.5*C_wind
        cost_A = np.array(cost_A)
        cost_A= cost_A.reshape(-1)
        cost_B = np.array(cost_B)
        cost_B= cost_B.reshape(-1)
        cost_C = np.array(cost_C)
        cost_C= cost_C.reshape(-1)
        #print('C成本:',sum(cost_C)+cost_of_Battery)
        #print('----------------------------------')
        new_row = pd.DataFrame({'E_B': [E_B], 'P_Battery_Max': [P_Battery_Max], 'cost': [sum(cost_A) + cost_of_Battery]})
        df_A = pd.concat([df_A, new_row], ignore_index=True)
        new_row = pd.DataFrame({'E_B': [E_B], 'P_Battery_Max': [P_Battery_Max], 'cost': [sum(cost_B) + cost_of_Battery]})
        df_B = pd.concat([df_B, new_row], ignore_index=True)
        new_row = pd.DataFrame({'E_B': [E_B], 'P_Battery_Max': [P_Battery_Max], 'cost': [sum(cost_C) + cost_of_Battery]})
        df_C = pd.concat([df_C, new_row], ignore_index=True)
        P_Battery_Max+=50
    
    E_B+=100


TypeError: Process() missing 1 required positional argument: 't'

In [ ]:
# 输出结果

print(df_A)
print(df_B)
print(df_C)

df_A.to_excel('./园区A_cost.xlsx')
df_B.to_excel('./园区B_cost.xlsx')
df_C.to_excel('./园区C_cost.xlsx')
# 画出E_B,P_Battery_Max与cost的关系
# 创建一个新的图形
# Create a 3D scatter plot
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] =False

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df_A['E_B'], df_A['P_Battery_Max'], df_A['cost'])

ax.set_xlabel('E_B')
ax.set_ylabel('P_Battery_Max')
ax.set_zlabel('cost')
plt.title('园区A E_B,P_Battery_Max与cost的关系')
plt.show()

# 保存图形到指定文件夹和文件名
# output_folder = 'problem1/pictures'
# output_filename = '园区A E_B,P_Battery_Max与cost的关系.png'
# output_path = f'{output_folder}/{output_filename}'
# plt.savefig(output_path, dpi=300)
# plt.close()


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df_B['E_B'], df_B['P_Battery_Max'], df_B['cost'])

ax.set_xlabel('E_B')
ax.set_ylabel('P_Battery_Max')
ax.set_zlabel('cost')
plt.title('园区B E_B,P_Battery_Max与cost的关系')
plt.show()

# # 保存图形到指定文件夹和文件名
# output_folder = 'problem1/pictures'
# output_filename = '园区B E_B,P_Battery_Max与cost的关系.png'
# output_path = f'{output_folder}/{output_filename}'
# plt.savefig(output_path, dpi=300)
# plt.close()


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df_C['E_B'], df_C['P_Battery_Max'], df_C['cost'])

ax.set_xlabel('E_B')
ax.set_ylabel('P_Battery_Max')
ax.set_zlabel('cost')
plt.title('园区C E_B,P_Battery_Max与cost的关系')
plt.show()

# # 保存图形到指定文件夹和文件名
# output_folder = 'problem1/pictures'
# output_filename = '园区C E_B,P_Battery_Max与cost的关系.png'
# output_path = f'{output_folder}/{output_filename}'
# plt.savefig(output_path, dpi=300)
# plt.close()

In [ ]:
# def charge(d_P):
#     global Battery
#     tmp=Battery
#     if P_Battery_Max >= d_P:
#         if Battery + d_P >= init_Battery_capacity_upper_bound:
#             Battery = init_Battery_capacity_upper_bound
#         else:
#             Battery += d_P
#     else:
#         if Battery + P_Battery_Max >= init_Battery_capacity_upper_bound:
#             Battery = init_Battery_capacity_upper_bound
#         else:
#             Battery += P_Battery_Max
#     #print('Battery充电:','delt_p:',d_P,' ',tmp,'->',Battery)
# def Cal_charge(P_wind,data,h):
#     index_lack=find_lack(data['Curt_C'],h)
#     if index_lack == -1:
#         print(f'从{h}电后面没有负荷缺口')
#         return 0

#     extra=sum(
#         data['Curt_C'][h:index_lack].apply(lambda x: P_Battery_Max if x > P_Battery_Max else x)
#         )
    
#     if extra>(SOC_UP-getSOC(t))*E_B:
        


#     lack=-data['Curt_C'][index_lack]

#     while lack<extra and index_lack<len(data) and data['Curt_C'][index_lack]<=0:
#         index_lack+=1
#         lack+=(-data['Curt_C'][index_lack])

#     if lack > extra:
#         Bu_Wind=lack-extra
#         if Bu_Wind>P_wind:
#             return P_wind
#         else:
#             return Bu_Wind
#     else:
#         return 0 